In [1]:
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import ContentType
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.types.message import ContentTypes
from aiogram.dispatcher import FSMContext
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from sqlalchemy import update
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.utils.callback_data import CallbackData
import typing


API_TOKEN = "5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY"


In [2]:
from food.paths import *
from food.search import *
import pandas as  pd
from food.psql import *
import pytz
timezones = pytz.all_timezones
import requests
from requests.structures import CaseInsensitiveDict
import urllib
from tzwhere import tzwhere

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
def geocode(q):
    geocoding_key = '5d96ac126bcb462cb373297924ab2cb4'
    url = "https://api.geoapify.com/v1/geocode/search?"

    params = {"apiKey":geocoding_key, 
              "text":q}

    resp = requests.get(url + urllib.parse.urlencode(params)).json()
    return  pd.json_normalize(resp['features']).sort_values('properties.rank.importance',ascending = False)[['properties.lat','properties.lon']].iloc[0].to_list()
   

In [5]:
def get_tz(q):
    lat,lon = geocode(q)
    return tzwhere.tzwhere().tzNameAt(lat,lon)

In [6]:
async def async_get_tz(q):
    return get_tz(q)

In [7]:
async def async_search_image(url, env='prod'):
    return search_image(url,env)

In [8]:
async def async_geocode(q):
    return geocode(q)

In [9]:
async def  async_insert_on_conflict(*args, **qwargs):
    return insert_on_conflict(*args, **qwargs)

In [10]:
async def add_sender(message):
    sender = message['from'].to_python()
    sender = pd.DataFrame(sender,index=[0]).drop(columns =['is_bot'])
    await async_insert_on_conflict(sender,'users',unique_cols=['id'])

In [11]:
def get_msg(query):
    dish = pd.read_sql(f"""select energy,protein,carb,fat from food.dishes 
                                where user_id={query['from']['id']} and 
                                message_id = {query['message']['message_id']}
                                order by id desc limit 1""",engine)
    plot_numtients = dish[['energy','protein','carb','fat']].reset_index(drop=True)
    plot_numtients.index = ['']
    return plot_numtients.astype(int).to_string()

In [12]:
def get_today_consumed(user_id):
    today_consumed = pd.read_sql(f"""select energy,grams,timestamp from {schema}.dishes
                                    where user_id = {user_id} and timestamp > now() - interval '24 hours'
                                    and grams is not null;""",engine).set_index("timestamp")
    today_consumed= today_consumed['energy']/100*today_consumed['grams']
    user_tz = engine.execute(f"""select value from food.user_properties 
                                where user_id={user_id} and
                                property='tz'
                                order by id desc limit 1""").first()

    user_tz = user_tz[0] if user_tz else 'UTC'
    today_consumed = today_consumed.tz_convert(user_tz)
    now = pd.Timestamp.now(tz = user_tz)
    today_consumed = today_consumed.reset_index()	
    this_morning = pd.Timestamp(year = now.year,month = now.month,day = now.day,hour = 3,tz = user_tz)
    today_consumed = today_consumed[today_consumed['timestamp'] > pd.Timestamp(this_morning)][0].sum()
    return int(today_consumed)

In [13]:

bot = Bot(token=API_TOKEN)
storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)

dishes_table = Dishes.__table__

add_dish_cb   = CallbackData('add dish', 'action')
measurment_cb = CallbackData('measurment', 'weight')
edit_dish_cb  = CallbackData('edit_dish', 'action')

ml_version = 0.2


In [14]:

set_timezone_command = types.BotCommand('set_timezone','set you timezone so that we know when your day starts')
commands = [set_timezone_command]
await bot.set_my_commands(commands)


True

In [15]:
def get_keyboard(t):
    markup = types.InlineKeyboardMarkup()
    if t == 'add dish' :  
        markup.add(types.InlineKeyboardButton('add dish', callback_data=add_dish_cb.new(action='yes')))
         
    elif t == 'measurment':

        markup = types.InlineKeyboardMarkup(row_width=8)
        btns_text = tuple([(str(p)) for p in range(10,510,10)])
        markup.add(*(types.InlineKeyboardButton(text, callback_data=measurment_cb.new(weight=text)) for text in btns_text))

    elif t == 'edit_dish':

        btns_text = ('remove','edit weight','add again')
        markup.add(*(types.InlineKeyboardButton(text, callback_data=edit_dish_cb.new(action=text)) for text in btns_text))


    

    return markup 


In [16]:
async def measurment(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    await query.answer()

    msg = query.to_python()['message']['text']
    msg = msg.split('|')[0] if '|' in msg else msg
    msg = f"{msg}"

    await bot.edit_message_text(
        msg,
        query.from_user.id,
        query.message.message_id,
        reply_markup=get_keyboard('measurment'),
    )

In [17]:
def get_update(query,grams):
    energy = engine.execute(f"""select energy from food.dishes 
                                    where user_id={query['from']['id']}
                                    and message_id = {query['message']['message_id']}
                                    order by id desc limit 1""").first()[0]
                                    #['reply_to_message']['message_id']
    stmt = (
                dishes_table.update()
                            .where(dishes_table.c.message_id == query['message']['message_id'])
                            .values(grams=grams)
                            .returning(dishes_table.c.id)
        )
    session.execute(stmt)
    session.commit()

    return int(energy)


In [18]:
#photo recieved
@dp.message_handler(content_types=ContentType.PHOTO)
async def process_photo(message: types.Message):
    await types.ChatActions.typing()

    await add_sender(message)

    photo  = message['photo'][-1]
    await photo.download(reference_images_path/photo['file_id'])
    image_url = await photo.get_url()
    dish = await async_search_image(url=image_url, env='prod')

    dish['photo_id']         = photo['file_id']
    dish['photo_message_id'] = message['message_id']
    sender = message['from'].to_python()
    dish['user_id'] = sender['id']
    dish['ml_version'] = ml_version 
    dish['timestamp']=pd.Timestamp.utcnow()

    
    plot_numtients = dish[['energy','protein','carb','fat']].reset_index(drop=True)
    plot_numtients.index = ['']
    
    msg = plot_numtients.astype(int).to_string()

    reply_message = await message.reply(msg, reply_markup=get_keyboard('add dish'))
    dish['message_id'] = reply_message['message_id']
    
    dish.to_sql('dishes',schema = schema,if_exists = 'append',index = False,con=engine)



   

In [19]:
@dp.message_handler(commands=['start'])
async def send_welcome(message: types.Message):
    
    await message.reply("""Counting calories as easy as taking pictures. Just capture everything before you eat it\n
                          Now send a photo of your meal to try""")

In [20]:
class CState(StatesGroup): 
    set_timezone   = State()

In [21]:
@dp.message_handler(commands=['set_timezone'])
async def send_welcome(message: types.Message, state: FSMContext):
    await CState.set_timezone.set()
    await message.reply(f"please search your town to set timezone")

In [22]:
@dp.message_handler(state=CState.set_timezone)
async def set_timezone(message: types.Message, state: FSMContext):
    await types.ChatActions.typing()
    await add_sender(message)
    tz = await async_get_tz(message.text)

    df = pd.DataFrame([[message['from']['id'],'tz',tz,pd.Timestamp.utcnow()]],columns = ['user_id','property','value','timestamp'])
    df.to_sql('user_properties',schema = schema,con = engine,if_exists = 'append',index = False)

    await state.finish()

    await message.reply(f"your tz is set to {tz}")


In [23]:
#add_dish pushed
@dp.callback_query_handler(add_dish_cb.filter(action=['yes']))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    await measurment(query, callback_data)

In [24]:
#add_dish pushed
@dp.callback_query_handler(edit_dish_cb.filter(action=['edit weight']))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    await measurment(query, callback_data)

In [25]:
#measure provided
@dp.callback_query_handler(measurment_cb.filter(weight=[str(p) for p in range(10,510,10)]))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    await query.answer()

    grams = int(callback_data['weight'])

    energy = get_update(query,grams)

    msg = query.to_python()['message']['text']
    msg = f"{msg}\n | consumed {grams} grams \n |  {int(energy/100*grams)} kcall \n today consumed {get_today_consumed(query['from']['id'])}"

    await bot.edit_message_text(
        msg,
        query.from_user.id,
        query.message.message_id,
        reply_markup=get_keyboard('edit_dish')
    )
    

In [26]:
#remove pushed
@dp.callback_query_handler(edit_dish_cb.filter(action=['remove']))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    _ = get_update(query,0)

    msg = query.to_python()['message']['text']
    msg = msg.split('|')[0] if '|' in msg else msg
    msg = f"{msg} \n | today consumed {get_today_consumed(query['from']['id'])}"

    await query.answer()
    await bot.edit_message_text(
        msg,
        query.from_user.id,
        query.message.message_id,
        reply_markup=get_keyboard('add dish'))

In [27]:
#add again pushed
@dp.callback_query_handler(edit_dish_cb.filter(action=['add again']))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):

    dish = pd.read_sql(f"""select description,energy,protein,carb,fat,score,photo_id,user_id,ml_version,photo_message_id
                from food.dishes 
                                                    where user_id={query['from']['id']}
                                                    and message_id = {query['message']['message_id']} limit 1""",engine)
    dish['timestamp'] = pd.Timestamp.utcnow()

    

    msg = query.to_python()['message']['text']
    msg = msg.split('|')[0] if '|' in msg else msg
    msg = f"{msg} \n | today consumed {get_today_consumed(query['from']['id'])}"

    await query.answer()
    message = await bot.send_message(chat_id=query['from']['id'], 
                                     reply_to_message_id = dish['photo_message_id'].iloc[0],
                                     text=msg, 
                                     reply_markup=get_keyboard('add dish'))

    dish['message_id'] = message['message_id']
    dish.to_sql('dishes',schema = schema,if_exists = 'append',index = False,con=engine)
    
    

In [28]:

if __name__ == '__main__':
    executor.start_polling(dp)

In [30]:
StatesGroup.

aiogram.dispatcher.filters.state.StatesGroup

In [55]:
types.Message(message_id=2069,chat = types.Chat(id=78882798)).text

In [24]:
types.Message??

Init signature:
types.Message(
    conf: 'typing.Dict[str, typing.Any]' = None,
    **kwargs: 'typing.Any',
) -> 'None'
Source:        
class Message(base.TelegramObject):
    """
    This object represents a message.

    https://core.telegram.org/bots/api#message
    """

    message_id: base.Integer = fields.Field()
    from_user: User = fields.Field(alias="from", base=User)
    sender_chat: Chat = fields.Field(base=Chat)
    date: datetime.datetime = fields.DateTimeField()
    chat: Chat = fields.Field(base=Chat)
    forward_from: User = fields.Field(base=User)
    forward_from_chat: Chat = fields.Field(base=Chat)
    forward_from_message_id: base.Integer = fields.Field()
    forward_signature: base.String = fields.Field()
    forward_date: datetime.datetime = fields.DateTimeField()
    is_automatic_forward: base.Boolean = fields.Field()
    reply_to_message: Message = fields.Field(base="Message")
    via_bot: User = fields.Field(base=User)
    edit_date: datetime.datetime = fields